In [1]:
import pytorch_lightning as pl

import numpy as np
import pandas as pd

import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from torchmetrics import AUROC

from sklearn.metrics import log_loss, roc_auc_score
from tqdm import tqdm

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

import warnings
warnings.filterwarnings('ignore')

## Анализ и подготовка данных

Колонки:

Признаки:

+ (banner):

banner_id - id баннера, где баннер - сама реклама
zone_id - id зоны, где зона - место на сайте для размещения рекламы
campaign_clicks - общее количество показов данной кампании (которой соотвествует баннер) данному юзеру, произошедшие до текущего показа. Кампанию стоит понимать как что-то общее (рекламодатель/тематика/ и т. п.) для баннеров.

+ (user):

oaid_hash - хэш юзера
os_id - id операционной системы
country_id - id страны

+ (date):

date_time - время показа рекламы (здесь возьму только час как категориальную фичу)

target:
clicks - был ли клик

удалить:
impressions - был ли показ (колонка с единицами)


_____
В предыдущем задании я кодировала все фичи с помощью One-hot encoding (кроме фичи campaign_clicks, ее логарифмирую и оставляю такой, какая она есть). В этот раз я сделаю почти то же самое - перекодирую все значения переменных на 0...N-1, где N- кол-во уникальных значений в колонке. Так как далее буду писать FFM на pytorch, вместо линейного слоя возьму Embedding, который как раз принимает такие значения.

In [2]:
# читаем данные
data = pd.read_csv('../data/data.csv')
# сразу удаляем ненужные колонки
data = data.drop(["banner_id0", "banner_id1", "rate0", "rate1", "g0", "g1", "coeff_sum0", "coeff_sum1"], axis=1)
# уточняем формат данных колокни с датой
data['date_time'] = pd.to_datetime(data['date_time'])
data.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,impressions,clicks
0,2021-09-27 00:01:30,0,0,5664530014561852622,0,0,0,1,1
1,2021-09-26 22:54:49,1,1,5186611064559013950,0,0,1,1,1
2,2021-09-26 23:57:20,2,2,2215519569292448030,3,0,0,1,1
3,2021-09-27 00:04:30,3,3,6262169206735077204,0,1,1,1,1
4,2021-09-27 00:06:21,4,4,4778985830203613115,0,1,0,1,1


На все колонки, кроме oaid_hash, уже смотрели. Поэтому в этот раз посмотрим только на нее.

In [3]:
oaid_counts = np.unique(data['oaid_hash'], return_counts=True)

In [4]:
# всего уникальных юзеров
len(oaid_counts[0])

6510316

In [5]:
# сколько всего юзеров, у которых больше 10 наблюдений?
np.sum(oaid_counts[1] > 10)

187834

Оставим лишь тех юзеров, у которых более менее достаточно наблюдений (например, 10). Остальным присвоим одинаковую категорию. Иначе, если оставлять всех юзеров, модель получается огромной и вряд ли сможет нормально обучиться (тем более для юзеров, у которых мало наблюдений)

In [6]:
# кодировщик категориальных колонок по типу OneHotEncoder,
# только просто перекодировывает значения категориальных переменных от 1 до N (где N - макс. число уник. значений)
# фиттим на трейне, для валидации и теста используем только трансформ
# такой вид кодировки был выбран, потому что создать таблицу с настоящим one-hot-encoding даже с большим кол-во оперативной памяти становится проблемой

class CatFeatureEncoder:
    def __init__(self):
        self.encod_dict = None
        self.nan_value = 0
        self.min_count = 10

    def fit(self, feature: pd.Series):
        unique_values, unique_counts = np.unique(feature, return_counts=True)

        # значение 0 будет использоваться для тех случаев в transform, когда значения нет в self.unique_values
        # тем, у кого меньше 20 наблюдений, тоже назначим категорию 0
        unique_values = unique_values[unique_counts >= self.min_count]
        self.encod_dict = {k: v + 1 for v, k in enumerate(unique_values)}

    def transform(self, feature: pd.Series):
        return feature.map(self.encod_dict).fillna(self.nan_value).astype(int)

    def fit_transform(self, feature: pd.Series):
        self.fit(feature)
        return self.transform(feature)

class ClicksDataset(Dataset):
    def __init__(self, df, encoders=None):
        data = df.copy()
        data.reset_index(inplace=True, drop=True)

        # создаем колонки час и день недели
        data['hour'] = data['date_time'].dt.hour
        # логарифмируем колонку campaign_clicks
        data['campaign_clicks'] = np.log(data['campaign_clicks']+0.001)
        # лейбл
        self.y = np.array(data['clicks'])
        data = data.drop(["clicks"], axis=1)
        # эти колонки будем использовать для обучения (кроме date и clicks)
        data = data[['hour', 'os_id', 'country_id', 'zone_id', 'banner_id', 'oaid_hash', 'campaign_clicks']]

        # вместо OneHotEncoding заменим во всех категориальных фичах значения на 1...N, где N- кол-во уникальных значений в колонке
        # в таком виде данные будут обрабатываться моделью
        cols_to_factorize = ['hour', 'os_id', 'country_id', 'zone_id', 'banner_id', 'oaid_hash']
        # если не передали encoder в init (для трейна) - обучаем его, фиттим на данные
        if encoders is None:
            self.encoders = [CatFeatureEncoder() for _ in range(len(cols_to_factorize))]
        # если передали (для вала и теста), то используем существующий
        else:
            self.encoders = encoders

        # кодируем все категориальные фичи
        X = []
        for i, col in enumerate(cols_to_factorize):
            enc = self.encoders[i]
            if encoders is None:
                X.append(enc.fit_transform(data[col]))
            else:
                X.append(enc.transform(data[col]))
        # присоединяем campaign_clicks отдельно
        X.append(np.array(data['campaign_clicks']))

        self.X = np.stack(X, axis=-1)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, item):
        y = self.y[item]
        x = self.X[item, :]
        return torch.from_numpy(x).type(torch.float32), torch.from_numpy(np.array(y)).type(torch.float32)

In [7]:
# возьмем в тест последний день, в валидацию - предпоследний день
# в трейн пойдет все остальные дни до них

train_data = data[data['date_time'].dt.date <= pd.Timestamp('2021-09-30').date()]#.sample(n=1500000, random_state=10)
val_data = data[data['date_time'].dt.date == pd.Timestamp('2021-10-01').date()]#.sample(n=500000, random_state=10)
test_data = data[data['date_time'].dt.date == pd.Timestamp('2021-10-02').date()]

print(len(train_data), len(val_data), len(test_data))

12049046 1643448 2128978


In [8]:
# подготовим датасеты для обучения и валидации
train_dataset = ClicksDataset(train_data)
val_dataset = ClicksDataset(val_data, train_dataset.encoders)

del train_data
del val_data

## Модель

Теперь напишем Field-Aware Factorization Machines.

Я тут помимо того, что обсуждали на лекции, еще ввожу какие-то дополнительные вещи для того, чтобы корректней обрабатывать интеракии категориальных фич с числовыми и для имплементации линейных слоев для категориальных и числовых фич.

Модель можно описать следующим образом:

$$y = \sigma(\text{LinearEmbLayer}(categ.feat.) + \text{Linear}(contin.feat.) + \text{FAFM(all features)})$$, где FAFM - FieldAwareFactorizationMachine  выглядит как:

$$\text{FAFM(all features)} = \text{FAFM(categ.feat. x categ. feat.)} + \text{FAFM(categ. feat. x cont. feat)} + (\text{FAFM(cont. feat. x cont. feat))}$$

(последнего слагаемого у нас нет, т.к. одна числовая фича

В $\text{FAFM(categ.feat. x categ. feat.)}$ все так, как обсуждалось на лекции - создаются эмбеддинги для интеракций со всеми другими фичами, перемножаются.

В $\text{FAFM(categ. feat. x cont. feat)}$ создаются еще эмбеддинги для категориальных фич для их взаимодейтсвия с количественной фичей, и просто умножаются на нее.

Размерность эмбеддингов везде равна 10.

Оптимизатор взяла AdaGrad по совету с лекции с дефолтными параметрами и weight_decay=5e-4 (L2 рег-ция).



In [9]:
# необходимые слои для FFM + сама модель FFM

class LinearEmbLayer(nn.Module):
    """
    Линейный слой, реализованный через nn.Embedding, для работы с категориальным фичами
    """
    def __init__(self, field_num_dims, output_dim=1):
        super().__init__()
        # nn.Embedding почти то же самое, что и nn.Linear
        #self.W = nn.Embedding(sum(field_num_dims), output_dim)
        self.W = nn.Embedding(sum(field_num_dims), output_dim)
        #self.b = nn.Parameter(torch.zeros((len(field_num_dims), output_dim)), requires_grad=True)
        # у меня проблемы с этим nn.Parameter - по какой-то причине, pytorch не сохраняет его в чекпойнт
        # он 100% оптимизируется, есть в параметрах модели, это проверяла
        # но при загрузке модели инициализируется с нуля, поэтому сразу на тесте метрики падают очень сильно
        # поэтому тут будет этот странный костыль с добавлением еще одного линейного слоя
        # (просто чтобы все-таки более честно изобразить линейный слой через слой nn.Embedding)
        # возможно в будущем разберусь, почему pytorch так себя ведет с слоями, инициализированными с помошью nn.Parameter
        # и тогда такой костыль будет не нужен. Но пока разобраться не получилось :(
        self.linear = nn.Linear(len(field_num_dims), output_dim)

    def forward(self, x):
        #return torch.sum(self.W(x) + self.b, dim=1)
        return self.linear(self.W(x).squeeze(2))

# с числовой переменной (у нас такая campaign_clicks), буду использовать обычный линейный слой


class FieldAwareFactorizationMachine(nn.Module):
    """
    слой для FFM (для обучения интеракций между фичами, разные эмбеддинги фичи для интеракций с разными фичами
    """
    def __init__(self, field_num_dims, embed_dim, cat_feats, cont_feats):
        super().__init__()
        self.cat_feats = cat_feats
        self.cont_feats = cont_feats
        self.num_fields = len(field_num_dims)
        # эмбеддинги для взаимодействия категориальных фичей с категориальными фичами
        # на позиции [i, j] находится эмбеддинг для взаимодействия j категории (среди всех значений всех категорий)
        # с эмбеддингом i признака
        self.cat_embeddings = torch.nn.ModuleList([
            torch.nn.Embedding(sum(field_num_dims), embed_dim) for _ in range(self.num_fields)
        ])
        # эмбеддинги для взаимодействия категориальных фичей с числовыми фичами
        self.cont_embeddings = torch.nn.ModuleList([
            torch.nn.Embedding(sum(field_num_dims), embed_dim) for _ in range(len(self.cont_feats))
        ])
        for embedding in self.cat_embeddings:
            torch.nn.init.xavier_uniform_(embedding.weight.data)
        for embedding in self.cont_embeddings:
            torch.nn.init.xavier_uniform_(embedding.weight.data)
        self.num_cat_values = sum(field_num_dims)

    def forward(self, x):
        # получаем эмбеддинги для всех значений всех категориальных фич
        xs = [self.cat_embeddings[i](x[:, self.cat_feats].type(torch.int)) for i in range(self.num_fields)]
        interactions = list()
        # считаем взаимодействия всех категориальных признаков
        for i in range(self.num_fields - 1):
            for j in range(i + 1, self.num_fields):
                interactions.append(xs[j][:, i] * xs[i][:, j])
        # взаимодействия всех числовых признаков со всем остальными
        # получаем эмбеддинги для всех значений всех категориальных фич (относительно числовых)
        xs = [self.cont_embeddings[i](x[:, self.cat_feats].type(torch.int)) for i in range(len(self.cont_feats))]
        # взамодействия между категор. и числовыми фичами
        for i in range(self.num_fields):
            for j in range(len(self.cont_feats)):
                interactions.append(xs[j][:, i] * x[:, j].view(-1, 1))
        # взамодействия между числовыми фичами
        for i in range(len(self.cont_feats)-1):
            for j in range(i + 1, len(self.cont_feats)):
                interactions.append(x[:, i] * x[:, j])

        interactions = torch.stack(interactions, dim=1)
        return interactions


class FFModel(nn.Module):
    def __init__(self, field_dims, embed_dim, cat_feats, cont_feats):
        """
        :param field_dims: список с кол-вом уникальных значений в категориальных фичах (в порядке этих фичей)
        :param embed_dim: размерность эмбеддингов
        :param cat_feats: индексы категориальных фичей
        :param cont_feats: индексы числовых фичей
        """
        super().__init__()
        # линейный слой для работы с категориальными фичами
        self.emb_linear = LinearEmbLayer(field_dims, 1)
        # линейный слой для работы с числовыми фичами
        self.linear = nn.Linear(len(cont_feats), 1)
        self.ffm = FieldAwareFactorizationMachine(field_dims, embed_dim, cat_feats, cont_feats)
        self.cat_feats = cat_feats
        self.cont_feats = cont_feats
        # далее 6 категориальных фичей как бы расплющиваются в одну, создавая одну большую категориальную фичу
        # для этого посчитаем сдвиги, которые надо прибавить к фичам, чтобы сохранить их различия
        self.offsets = np.array((0, *np.cumsum(len(field_dims))[:-1]), dtype=np.long)

    def forward(self, x):
        x[:, self.cat_feats] = x[:, self.cat_feats] + x.new_tensor(self.offsets).unsqueeze(0)
        ffm = torch.sum(torch.sum(self.ffm(x), dim=1), dim=1, keepdim=True)
        x = self.linear(x[:, self.cont_feats]) + self.emb_linear(x[:, self.cat_feats].type(torch.int)) + ffm
        return torch.sigmoid(x.squeeze(1))

In [10]:
# pytorch Lightning модель для обучения
class ClicksModel(pl.LightningModule):
    def __init__(self, field_dims, embed_dim, cat_feats, cont_feats):
        super().__init__()
        self.model = FFModel(field_dims, embed_dim, cat_feats, cont_feats)

    def forward(self, X):
        return self.model(X)

    def training_step(self, batch, batch_nb):
        x, y = batch
        loss = nn.BCELoss()
        preds = self(x)
        return {'loss': loss(preds, y), 't_preds': preds, 't_true': y}

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        roc = AUROC(num_classes=2).to(self.device)
        all_preds = torch.hstack([x['t_preds'] for x in outputs])
        all_targets = torch.hstack([x['t_true'] for x in outputs])
        avg_roc = roc(all_preds, all_targets.type(torch.int))
        print(f"Epoch {self.trainer.current_epoch}, Train_loss: {round(float(avg_loss), 3)}, "
              f"Train_roc_auc: {round(float(avg_roc), 3)}")
        self.log('train_loss', avg_loss, prog_bar=True, on_epoch=True, on_step=False)
        self.log('train_roc_auc', avg_roc, prog_bar=True, on_epoch=True, on_step=False)

    def configure_optimizers(self):
        """ Define optimizers and LR schedulers. """
        # optimizer = torch.optim.Adam(self.parameters(), lr=1e-3, weight_decay=5e-4)
        optimizer = torch.optim.Adagrad(self.parameters(), weight_decay=5e-4)
        lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                                  mode='max',
                                                                  factor=0.2,
                                                                  patience=5,
                                                                  verbose=True)
        lr_dict = {
            "scheduler": lr_scheduler,
            "interval": "epoch",
            "frequency": 1,
            "monitor": "val_roc_auc"
        }

        return [optimizer], [lr_dict]

    def validation_step(self, batch, batch_idx):
        """the full validation loop"""
        x, y = batch
        ce_loss = nn.BCELoss()
        preds = self(x).detach()
        loss = ce_loss(preds, y)
        return {'val_loss': loss, 'v_preds': preds, 'v_true': y}

    def validation_epoch_end(self, outputs):
        """log and display average val loss and val_f1"""
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        roc = AUROC(num_classes=2).to(self.device)
        all_preds = torch.hstack([x['v_preds'] for x in outputs])
        all_targets = torch.hstack([x['v_true'] for x in outputs])
        val_roc = roc(all_preds, all_targets.type(torch.int))
        print(f"Epoch {self.trainer.current_epoch}, Val_loss: {round(float(avg_loss), 3)}, "
              f"Val_roc_auc: {round(float(val_roc), 3)}")
        self.log('val_loss', avg_loss, prog_bar=True, on_epoch=True, on_step=False)
        self.log('val_roc_auc', val_roc, prog_bar=True, on_epoch=True, on_step=False)


In [11]:
# функция для обучения модели

def train():

    MyModelCheckpoint = ModelCheckpoint(dirpath='checkpoints/',
                                        filename='best_model',
                                        monitor='val_roc_auc',
                                        mode='max',
                                        save_top_k=1)


    MyEarlyStopping = EarlyStopping(monitor="val_roc_auc",
                                    mode="max",
                                    patience=10,
                                    verbose=True)

    # колонки ['hour', 'os_id', 'country_id', 'zone_id', 'banner_id', 'oaid_hash']

    field_dims = np.max(train_dataset.X[:, :-1], axis=0).astype(int) + 1
    cat_feats = np.arange(train_dataset.X.shape[1]-1)
    cont_feats = [train_dataset.X.shape[1]-1]
    # размер эмбеддингов будет равен 10
    model = ClicksModel(field_dims, 10, cat_feats, cont_feats)

    train_loader = DataLoader(train_dataset, shuffle=True, batch_size=2048)
    val_loader = DataLoader(val_dataset, shuffle=False, batch_size=2048)

    trainer = pl.Trainer(
        max_epochs=50,
        gpus=1,
        callbacks=[MyEarlyStopping, MyModelCheckpoint],
        log_every_n_steps=1
    )
    trainer.fit(model, train_loader, val_loader)

In [12]:
train()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | FFModel | 11.7 M
----------------------------------
11.7 M    Trainable params
0         Non-trainable params
11.7 M    Total params
46.676    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Epoch 0, Val_loss: 0.471, Val_roc_auc: 0.52


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Epoch 24, Val_loss: 0.154, Val_roc_auc: 0.773
Epoch 24, Train_loss: 0.095, Train_roc_auc: 0.772


Validation: 0it [00:00, ?it/s]

Monitored metric val_roc_auc did not improve in the last 10 records. Best score: 0.773. Signaling Trainer to stop.


Epoch 25, Val_loss: 0.154, Val_roc_auc: 0.773
Epoch 25, Train_loss: 0.095, Train_roc_auc: 0.772


## Оценка модели на тесте

In [26]:
test_dataset = ClicksDataset(test_data, train_dataset.encoders)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=512, num_workers=4)

field_dims = np.max(train_dataset.X[:, :-1], axis=0).astype(int) + 1
cat_feats = np.arange(train_dataset.X.shape[1]-1)
cont_feats = [train_dataset.X.shape[1]-1]
model = ClicksModel.load_from_checkpoint('checkpoints/best_model.ckpt', field_dims=field_dims, embed_dim=10, cat_feats=cat_feats, cont_feats=cont_feats)

model.eval()

y_true = []
predictions = []

for x, y in tqdm(test_loader):
    predictions.extend(model(x).detach().cpu().numpy().tolist())
    y_true.extend(y.cpu().numpy().tolist())

ll = log_loss(y_true, predictions)
roc = roc_auc_score(y_true, predictions)

print(f"Log loss: {ll}, Roc auc: {roc}")

100%|██████████| 4159/4159 [00:11<00:00, 374.96it/s]


Log loss: 0.13859916958197083, Roc auc: 0.7466247598149071


Получилось как-то не так уж хорошо, как могло быть (сравнивая с результатами с пред.домашки, где модель была попроще). Хотя, конечно, и лучше бейзлайна.

Мои предположения - у меня есть ошибки в имплементации алгоритма (плюс я там сама еще придумала способы работы с числовыми фичами, и не уверена, что правильно придумала)

Плюс другое предположение, которое проще проверить: из-за того, что урезали кол-во значение в колонке с идентификаторами пользователя, могло оказаться, что в тесте в этой колонке слишком много нулей (так как заполняла нулями значения тех юзеров, которые были не представлены или представлены слишком мало в датасете). Вообще говоря, это значит, что для модели это все одно значение, что возможно могло плохо сказаться на ее работе.

Посмотрим же на распределения значений в этой колонке во всех частях датасета

In [19]:
np.unique(train_dataset.X[:, 5], return_counts=True)

(array([0.00000e+00, 1.00000e+00, 2.00000e+00, ..., 1.61559e+05,
        1.61560e+05, 1.61561e+05]),
 array([8497256,      18,      10, ...,      10,      27,      13]))

In [20]:
np.unique(val_dataset.X[:, 5], return_counts=True)

(array([0.00000e+00, 1.00000e+00, 5.00000e+00, ..., 1.61558e+05,
        1.61560e+05, 1.61561e+05]),
 array([1422091,       2,      15, ...,      21,       6,       1]))

In [21]:
np.unique(test_dataset.X[:, 5], return_counts=True)

(array([0.00000e+00, 3.00000e+00, 5.00000e+00, ..., 1.61555e+05,
        1.61558e+05, 1.61560e+05]),
 array([1920043,       2,      15, ...,       3,       2,       4]))

Грустная картина получается, нулей много абсолютно везде, возможно использование этой колонки вообще потеряло смысл. Но использовать ее полностью, без урезания кол-во возможных значений, у меня не получилось - получаются огроменные эмбеддинги, и модель получается тоже огромной. Интересно было бы послушать, как все-таки правильно работать с такого рода категориальными переменными, которые имеют очень много значений, в моделях, подобных FFM.

Я бы попробовала сравнить свои результаты с результатами какой-нибудь готовой имплементации FFM, но на это, к сожалению, уже совсем нет времени.